In [1]:
from urllib2 import urlopen
from urllib2 import HTTPError, URLError
import re
import urllib2
from bs4 import BeautifulSoup
from time import sleep
import unicodecsv as csv
import os
import random

In [2]:
def make_soup(url):
    hdr = {"User-Agent": "Mozilla/5.0"}
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=hdr)).read(),'lxml')

In [31]:
def get_review(url,startID):
    soup = make_soup(url)
    divtable = soup.find("div", class_ = "review_table")
    reviewlist = divtable.find_all("div", class_ = "row review_table_row")
    completereviewdict = {}
    for x in reviewlist: 
        completereviewdict[str(startID)] = {}
        leftcolumn = x.find("div", class_ = "col-sm-13 col-xs-24 col-sm-pull-4 critic_name")
        critic = leftcolumn.find("a", class_ = "unstyled bold articleLink").get_text()
        journal = leftcolumn.find("a", class_ = "subtle").get_text()
        rightcolumn = x.find("div", class_ = "col-xs-16 review_container")
        date = rightcolumn.find("span", class_ = "review_date subtle small").get_text() 
        # need to add a if else loop for fresh/rotten icons. . . if "review_icon icon small fresh" - return "Fresh" 
        criticreviewdesc = rightcolumn.find("div", class_ = "the_review").get_text().strip()
        reviewdict = {"Critic":critic, "Journal": journal, "Date":date, "Critic Review Description":criticreviewdesc}
        completereviewdict[str(startID)].update(reviewdict)
        startID += 1
    return completereviewdict   

In [32]:
bigdict = {}
for x in range(11,100): # Change ending index (exclusive) accordingly
    url = "https://www.rottentomatoes.com/m/redacted/reviews/?sort="+str(x)+"&type=user" # Change movie url accordingly
    pageStartID = (x-1)*20 + 1
    onePageDict = get_review(url,pageStartID)
    bigdict.update(onePageDict)
    rdtime = random.uniform(0.5,2)
    sleep(rdtime)

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
len(bigdict) # Check how many reviews were scraped.

In [ ]:
os.chdir("/Users/tuh48546/Desktop/Critic Reviews/") # Change directory accordingly

In [ ]:
with open("Redacted_1To100_Mar04.csv", "w") as toWrite: # Change file name accordingly
    writer = csv.writer(toWrite, delimiter=',')
    writer.writerow(["Review ID", "Date", "Journal",
    "Critic", 'Critic Review Description'])
    for a in bigdict.keys():
        try:
            writer.writerow([a,
                            bigdict[a]["Date"].encode('utf-8'),
                            bigdict[a]["Journal"].encode('utf-8'),
                            bigdict[a]["Critic"].encode('utf-8'),
                            bigdict[a]["Critic Review Description"].encode('utf-8')])
        except UnicodeDecodeError as detail:
            print 'UnicodeDecodeError'
            print a
            print detail
            continue
        except AttributeError as detail:
            print 'AttributeError'
            print a
            print detail
            continue
print 'Done writing file.'